# Initial submission notebook

Working out the submission part of the pipeline with a very simple example

In [ ]:
import pandas as pd

import enefit
env = enefit.make_env()
iter_test = env.iter_test()

## Very simple prediction method for now:

Use the equivalent datapoint from the previous day to predict today.

In [ ]:
def makePredictions(test, revealed_targets):
    test = test.assign(time=pd.to_datetime(test.prediction_datetime).dt.time)
    revealed_targets = revealed_targets.assign(
                        time=pd.to_datetime(revealed_targets.datetime).dt.time
                        )
        
    id_keys = ['time', 'prediction_unit_id', 'is_consumption']
    merge_id_key = test[id_keys+['row_id']].merge(
                            revealed_targets[id_keys+['target']],
                            on=id_keys, how="left"
                            )
    return merge_id_key.fillna(0.0).sort_values('row_id')

In [ ]:
counter = 0

for (test, revealed_targets, client, weatherHistorical, weatherForecast,
    electricityPrices, gasPrices, sample_prediction) in iter_test:

    # requirements before we make changes
    required_dtypes = sample_prediction.dtypes
    required_shape = sample_prediction.shape
    required_rows = pd.DataFrame()
    required_rows.loc[:, 'row_id'] = sample_prediction.row_id

    
    predictions = makePredictions(test, revealed_targets)
    
    sample_prediction.loc[:, 'target'] = predictions.loc[: ,'target']
    
    # ensure assumptions about the data and submissions
    if (test.row_id != sample_prediction.row_id).any():
        raise Exception('test and sample_prediction mismatch')
    
    if (predictions.row_id != sample_prediction.row_id).any():
        raise Expection('Missing or misaligned rows in the predictions')
    
    if sample_prediction.isna().any().any():
        raise Exception('Nan values in the submission')

    if (sample_prediction.dtypes != required_dtypes).any():
        raise Exception('The submission changes the data types')

    if (sample_prediction.shape != required_shape):
        raise Exception('The submission changes the shape')
        
    if (sample_prediction.row_id != required_rows.row_id).any():
        raise Exception('The submission changes the row ids')
    
    env.predict(sample_prediction)
    counter += 1
    

In [ ]:
sample_prediction